## CHAPTER 10. 데이터베이스 자료처리

DB: 필요한 자료를 중복 최소화하여 컴퓨터 기억장치 내 모아둔 집합체를 말함. 통합, 저장, 운영

DBMS 데이터베이스관리시스템 ex) MySQL, Oracle, SQLite

## 10.1 관계형데이터베이스  & 10.2 SQLite3

2차원 테이블 형태로 제공하는 데이터베이스

이를 시스템에 구축하기 위해서 데이터베이스관리시스템(DBMS) 소프트웨어 이용


1) DBMS

사용자 인터페이스 + 데이터베이스언어(SQL)로 구성됨 

ex) Oracle, MySQL(MariaDB), MS-SQL, Infomix, MongoDB, SQLite3 등

주요기능
- 정의(definition): 자료의 형태, 구조, 저장에 관한 내용
- 조작(manipulation): 사용자의 요구에따른 검색, 수정, 삽입, 삭제 지원
- 제어(control): 데이터 무결성 유지, 보안, 병행수여 제어 등의 기능 제공



2) SQLitie3  (MySQL과 비슷)

기기 내부에서만 사용할 수 있는 DBMS. 주로 모바일, 소형기기에서 DBMS를 생성하는 SW.

파이썬에서는 import를 통해 바로 사용 가능

① 테이블만들기

[타입] text: 문자형 / numeric: 숫자형 / integer: 정수형 / real: 실수형 / blob: 대용량자료

[제약조건] primary key: 기본키 / unique: 중복불가 / not null: 생략불가 / check: 제한조건 / defuault: 기본값

예) Create table if not exists test(
    name text(10), primary key,
    phone text(15) not null, 
    addr text(50)
    )


② CRUD 만들기   Create, Read, Update, Delete  (조작 기능)

정보 참조, 검색, 갱신을 가리키기도 함

INSERT into table-name (column-name) values(값1, 값2,..);
SELECT column-name or * from table-name where 조건식;
UPDATE table-name set column-name = 수정값 where 조건식;
DELETE from table-name where 조건식;


In [1]:
import sqlite3
print(sqlite3.sqlite_version_info)

(3, 35, 5)


### SQLite3 데이터베이스 만들기 예 (p289)

In [6]:
try:
    # (1) DB연동 객체 : 지정 경로에 DB파일 생성, 해당 DB로 연결되는 객체(conn)가 생성됨
    conn = sqlite3.connect("Data/sqlite_db")   # DB생성 > 연결 object
    
    # (2) SQL실행 객체 : cursor 호출하여 sql문 실행 객체 생성 
    ##### cursor: 쿼리문에 의해 반환되는 결과값 저장하는 메모리공간
    cursor = conn.cursor()
    
    # (3) table 생성 : test_table을 생성
    sql = 'create table if not exists test_table(name text(10), phone text(15), addr text(50))'
    cursor.execute(sql)   # cursor객체에서 실행할 수 있는 execute함수로 실행
    
    # (4) 레코드 추가 : insert문으로 레코드 추가
    cursor.execute("insert into test_table values('홍길동','010-1111-1111','서울시')")
    cursor.execute("insert into test_table values('이순신','010-2222-2222','하남시')")
    cursor.execute("insert into test_table values('강감찬','010-1111-1111','평양시')")
    
    # (5) 레코드 조회 : select문 형식으로 레코드 조회
    cursor.execute("select * from test_table")
    rows = cursor.fetchall()    # 조회 레코드 가져오기
    ##### fetch: 커서에서 원하는 결과값을 추출
    
    # (6) 레코드 출력1 : 한 레코드씩 튜플 단위로 반환
    for row in rows:
        print(row)
        
    # (7) 레코드 출력2 : 전체 레코드를 컬럼 단위로 출력
    print('이름 \t전화번호 \t주소')       # Escape Code: \n: 줄바꿈, \t: tab, \0: 공백, \'\'
    for row in rows:
        print(row[0],'\t',row[1],'\t',row[2])

except Exception as e:
    print("DB연동 실패: ", e)
    conn.rollback()    # 실행취소

finally:
    cursor.close()   # cursor 객체 닫기
    conn.close()     # conn 객체 닫기

('홍길동', '010-1111-1111', '서울시')
('이순신', '010-2222-2222', '하남시')
('강감찬', '010-1111-1111', '평양시')
이름 	전화번호 	주소
홍길동 	 010-1111-1111 	 서울시
이순신 	 010-2222-2222 	 하남시
강감찬 	 010-1111-1111 	 평양시


### SQLite3 CRUD 예 (p291~p292)

In [13]:
try: 
    # DB 연동
    conn = sqlite3.connect("Data/sqlite_crud_db")   # DB생성 > 연결 object
    
    # SQL실행 객체 :
    cursor = conn.cursor()
    
    # (1) table 생성 
    sql = """create table if not exists goods(
        code integer primary key,
        name text(30) unique not null,
        su integer default 0,
        dan real default 0.0
        )"""
    cursor.execute(sql)   # sql실행
    
    # (2) 레코드 추가
    cursor.execute("insert into goods values(1, '냉장고', 2, 8500000)")
    cursor.execute("insert into goods values(2, '세탁기', 3, 5500000)")
    cursor.execute("insert into goods(code, name) values(3, '전자레인지')")
    cursor.execute("insert into goods(code, name, dan) values(4, 'HDTV', 15000000)")
    conn.commit()   # DB 반영
    
    # (3) 레코드 조회 
    sql = "select * from goods"
    cursor.execute(sql)
    rows = cursor.fetchall()    # 조회 레코드 가져오기
    
    for row in rows:
        print(row[0], row[1], row[2], row[3])
    print('검색된 레코드 수: ', len(rows))
        
    # (4) 상품명 조회
    name = input('\n상품명 입력: ')
    sql = f"select * from goods where name like '%{name}%'"
    cursor.execute(sql)
    rows = cursor.fetchall() 
    
    if rows:  # null = False
        for row in rows:
            print(row)
    else: 
        print('검색된 레코드 없음')

except Exception as e:
    print("DB연동 ERROR: ", e)
    conn.rollback()    

finally:
    cursor.close()   
    conn.close()     

1 냉장고 2 8500000.0
2 세탁기 3 5500000.0
3 전자레인지 0 0.0
4 HDTV 0 15000000.0
검색된 레코드 수:  4

상품명 입력: 냉장고
(1, '냉장고', 2, 8500000.0)


### SQLite3 CRUD 예 (p291~p292) 레코드 추가

In [17]:
try: 
    # DB 연동
    conn = sqlite3.connect("Data/sqlite_crud_db")  
    
    # SQL실행 객체 :
    cursor = conn.cursor()
    
    # (1) table 생성 
#     sql = """create table if not exists goods(
#         code integer primary key,
#         name text(30) unique not null,
#         su integer default 0,
#         dan real default 0.0
#         )"""
#     cursor.execute(sql)   # sql실행
    
    # (2) 레코드 추가
#     cursor.execute("insert into goods values(1, '냉장고', 2, 8500000)")
#     cursor.execute("insert into goods values(2, '세탁기', 3, 5500000)")
#     cursor.execute("insert into goods(code, name) values(3, '전자레인지')")
#     cursor.execute("insert into goods(code, name, dan) values(4, 'HDTV', 15000000)")
#     conn.commit()   # DB 반영
    
    # (5) 레코드 추가: 2차레코드 조회 
    code = int(input('code 입력: '))
    name = input('name 입력: ')
    su = int(input('su 입력: '))
    dan = int(input('dan 입력: '))
    
    sql = f"insert into goods values({code}, '{name}', {su}, {dan})"
    cursor.execute(sql)
    conn.commit()
    
    # (3) 레코드 조회 
    sql = "select * from goods"
    cursor.execute(sql)
    rows = cursor.fetchall()    # 조회 레코드 가져오기
    
    for row in rows:
        print(row[0], row[1], row[2], row[3])
    print('검색된 레코드 수: ', len(rows))
           
    # (4) 상품명 조회
    name = input('\n상품명 입력: ')
    sql = f"select * from goods where name like '%{name}%'"
    cursor.execute(sql)
    rows = cursor.fetchall() 
    
    if rows:  # null = False
        for row in rows:
            print(row)
    else: 
        print('검색된 레코드 없음')

except Exception as e:
    print("DB연동 ERROR: ", e)
    conn.rollback()    

finally:
    cursor.close()   
    conn.close()     

code 입력: 5
name 입력: 건조기
su 입력: 1
dan 입력: 45000000
1 냉장고 2 8500000.0
2 세탁기 3 5500000.0
3 전자레인지 0 0.0
4 HDTV 0 15000000.0
5 건조기 1 45000000.0
검색된 레코드 수:  5
상품명 입력: 건조기
(5, '건조기', 1, 45000000.0)


### SQLite3 CRUD 예 (p291~p292) 레코드 수정

In [26]:
try: 
    # DB 연동
    conn = sqlite3.connect("Data/sqlite_crud_db")   
    
    # SQL실행 객체 :
    cursor = conn.cursor()
    
    # (1) table 생성 
#     sql = """create table if not exists goods(
#         code integer primary key,
#         name text(30) unique not null,
#         su integer default 0,
#         dan real default 0.0
#         )"""
#     cursor.execute(sql)   # sql실행
    
    # (2) 레코드 추가
#     cursor.execute("insert into goods values(1, '냉장고', 2, 8500000)")
#     cursor.execute("insert into goods values(2, '세탁기', 3, 5500000)")
#     cursor.execute("insert into goods(code, name) values(3, '전자레인지')")
#     cursor.execute("insert into goods(code, name, dan) values(4, 'HDTV', 15000000)")
#     conn.commit()   # DB 반영
    
    # (5) 레코드 추가: 2차레코드 조회 
#     code = int(input('code 입력: '))
#     name = input('name 입력: ')
#     su = int(input('su 입력: '))
#     dan = int(input('dan 입력: '))
    
#     sql = f"insert into goods values({code}, '{name}', {su}, {dan})"
#     cursor.execute(sql)
#     conn.commit()
    
    # (6) 레코드 수정: code >> su, dan 수정
    code = int(input('수정하려는 code 입력: '))
    su = int(input('수정내용 su 입력: '))
    dan = int(input('수정내용 dan 입력: '))
    
    sql = f"update goods set su = {su}, dan = {dan} where code = {code}"
    cursor.execute(sql)
    conn.commit()
    
    # (3) 레코드 조회 
    sql = "select * from goods"
    cursor.execute(sql)
    rows = cursor.fetchall()    # 조회 레코드 가져오기
    
    for row in rows:
        print(row[0], row[1], row[2], row[3])
    print('검색된 레코드 수: ', len(rows))
           
    # (4) 상품명 조회
#     name = input('\n상품명 입력: ')
#     sql = f"select * from goods where name like '%{name}%'"
#     cursor.execute(sql)
#     rows = cursor.fetchall() 
    
#     if rows:  # null = False
#         for row in rows:
#             print(row)
#     else: 
#         print('검색된 레코드 없음')

except Exception as e:
    print("DB연동 ERROR: ", e)
    conn.rollback()    

finally:
    cursor.close()   
    conn.close()     

수정하려는 code 입력: 3
수정내용 su 입력: 7
수정내용 dan 입력: 1200000
1 냉장고 0 6000000.0
2 세탁기 2 3800000.0
3 전자레인지 7 1200000.0
4 HDTV 0 15000000.0
5 건조기 1 45000000.0
검색된 레코드 수:  5


### SQLite3 CRUD 예 (p291~p292) 레코드삭제

In [27]:
try: 
    # DB 연동
    conn = sqlite3.connect("Data/sqlite_crud_db")   # DB생성 > 연결 object
    
    # SQL실행 객체 :
    cursor = conn.cursor()
    
    # (1) table 생성 
#     sql = """create table if not exists goods(
#         code integer primary key,
#         name text(30) unique not null,
#         su integer default 0,
#         dan real default 0.0
#         )"""
#     cursor.execute(sql)   # sql실행
    
    # (2) 레코드 추가
#     cursor.execute("insert into goods values(1, '냉장고', 2, 8500000)")
#     cursor.execute("insert into goods values(2, '세탁기', 3, 5500000)")
#     cursor.execute("insert into goods(code, name) values(3, '전자레인지')")
#     cursor.execute("insert into goods(code, name, dan) values(4, 'HDTV', 15000000)")
#     conn.commit()   # DB 반영
    
    # (5) 레코드 추가: 2차레코드 조회 
#     code = int(input('code 입력: '))
#     name = input('name 입력: ')
#     su = int(input('su 입력: '))
#     dan = int(input('dan 입력: '))
    
#     sql = f"insert into goods values({code}, '{name}', {su}, {dan})"
#     cursor.execute(sql)
#     conn.commit()
    
    # (6) 레코드 수정: code >> su, dan 수정
#     code = int(input('수정하려는 code 입력: '))
#     su = int(input('수정내용 su 입력: '))
#     dan = int(input('수정내용 dan 입력: '))
    
#     sql = f"update goods set su = {su}, dan = {dan} where code = {code}"
#     cursor.execute(sql)
#     conn.commit()
    
    # (7) 레코드 삭제: 
    code = int(input('삭제 code 입력(1,2,3,4,5 중 하나): '))
    sql = f"delete from goods where code = {code}"
    cursor.execute(sql)
    conn.commit()
    
    # (3) 레코드 조회 
    sql = "select * from goods"
    cursor.execute(sql)
    rows = cursor.fetchall()    # 조회 레코드 가져오기
    
    for row in rows:
        print(row[0], row[1], row[2], row[3])
    print('검색된 레코드 수: ', len(rows))
           
    # (4) 상품명 조회
    name = input('상품명 입력: ')
    sql = f"select * from goods where name like '%{name}%'"
    cursor.execute(sql)
    rows = cursor.fetchall() 
    
    if rows:  # null = False
        for row in rows:
            print(row)
    else: 
        print('검색된 레코드 없음')

except Exception as e:
    print("DB연동 ERROR: ", e)
    conn.rollback()    

finally:
    cursor.close()   
    conn.close()     

삭제 code 입력(1,2,3,4,5 중 하나): 5
1 냉장고 0 6000000.0
2 세탁기 2 3800000.0
3 전자레인지 7 1200000.0
4 HDTV 0 15000000.0
검색된 레코드 수:  4
상품명 입력: 
(1, '냉장고', 0, 6000000.0)
(2, '세탁기', 2, 3800000.0)
(3, '전자레인지', 7, 1200000.0)
(4, 'HDTV', 0, 15000000.0)


In [29]:
try: 
    # DB 연동
    conn = sqlite3.connect("Data/sqlite_crud_db")   # DB생성 > 연결 object
    
    # SQL실행 객체 :
    cursor = conn.cursor()
    
    # (1) table 생성 
#     sql = """create table if not exists goods(
#         code integer primary key,
#         name text(30) unique not null,
#         su integer default 0,
#         dan real default 0.0
#         )"""
#     cursor.execute(sql)   # sql실행
    
    # (2) 레코드 추가
#     cursor.execute("insert into goods values(1, '냉장고', 2, 8500000)")
#     cursor.execute("insert into goods values(2, '세탁기', 3, 5500000)")
#     cursor.execute("insert into goods(code, name) values(3, '전자레인지')")
#     cursor.execute("insert into goods(code, name, dan) values(4, 'HDTV', 15000000)")
#     conn.commit()   # DB 반영
    
    # (5) 레코드 추가: 2차레코드 조회 
#     code = int(input('code 입력: '))
#     name = input('name 입력: ')
#     su = int(input('su 입력: '))
#     dan = int(input('dan 입력: '))
    
#     sql = f"insert into goods values({code}, '{name}', {su}, {dan})"
#     cursor.execute(sql)
#     conn.commit()
    
    # (6) 레코드 수정: code >> su, dan 수정
#     code = int(input('수정하려는 code 입력: '))
#     su = int(input('수정내용 su 입력: '))
#     dan = int(input('수정내용 dan 입력: '))
    
#     sql = f"update goods set su = {su}, dan = {dan} where code = {code}"
#     cursor.execute(sql)
#     conn.commit()
    
    # (7) 레코드 삭제: 
#     code = int(input('삭제 code 입력(1,2,3,4,5 중 하나): '))
#     sql = f"delete from goods where code = {code}"
#     cursor.execute(sql)
#     conn.commit()
    
    # (3) 레코드 조회 
    sql = "select * from goods"
    cursor.execute(sql)
    rows = cursor.fetchall()    # 조회 레코드 가져오기
    print(rows)

except Exception as e:
    print("DB연동 ERROR: ", e)
    conn.rollback()    

finally:
    cursor.close()   
    conn.close()     

[(1, '냉장고', 0, 6000000.0), (2, '세탁기', 2, 3800000.0), (3, '전자레인지', 7, 1200000.0), (4, 'HDTV', 0, 15000000.0)]
